# Descomposición por valores singulares y sistemas de recomendación.

Al igual que el análisis de componentes principales (PCA por sus siglas en inglés), la descomposición por valores singulares (SVD en inglés) es uno de los métodos más usuales para reducir la dimensión de nuestro conjunto de datos.

A diferencia del PCA, este método no requiere del uso de la matriz de **varianzas y covarianzas**.

+ **A favor:** Reduce la dimensión de nuestros datos, remueve el ruido que podrían contener.
+ **En contra:** Se pierde interpretación en los datos transformados.
+ **Tipo de datos:** Numéricos.

En estas notas, se expondrá la relación entre SVD y los sistemas de recomendación.
***

## Descomposición de la matriz de datos.
Sea $\mathbf{D_{m \times n}}$ una matriz de $m \times n$ la cual representa nuestro conjunto de datos. El método de descomposición por valores singulares nos dice que:

$$ \mathbf{D_{m \times n}} = \mathbf{U_{m \times m}} \mathbf{\Sigma_{m \times n}} \mathbf{V_{n \times n}^{T}}$$

en donde la matriz $\mathbf{\Sigma_{m \times n}}$ es una matriz con elementos igual a $0$, excepto posiblemente, en su diagonal. Estos valores distintos de cero, reciben el nombre de **valores singulares** y son igual a la raiz cuadrada de los eigenvalores de la matriz $\mathbf{D_{m \times n}} \mathbf{D_{m \times n}^{T}}$. Además, los valores en la diagonal están ordenados de mayor a menor.

## SVD en Python con numpy.
Es posible obtener factorización anterior utilizando el paquete **numpy**, en particular, la función **svd** del módulo **linalg**.

In [1]:
import numpy as np
datos = np.matrix([[1,2,3], [4,5,6]])
num_renglones = datos.shape[0] # m
num_columnas = datos.shape[1] #n
U, sig, VT = np.linalg.svd(datos)

print('m es igual a ' + str(num_renglones))
print('n es igual a ' + str(num_columnas))
print('La dimensión de U es ' + str(U.shape))
print('La dimensión de sigma es ' + str(sig.shape))
print('La dimensión de V^T es ' + str(VT.shape))
print(sig) #No es una matriz!

m es igual a 2
n es igual a 3
La dimensión de U es (2, 2)
La dimensión de sigma es (2,)
La dimensión de V^T es (3, 3)
[9.508032   0.77286964]


Como podemos observar, la función **svd**, en lugar de regresar la matriz $\mathbf{\Sigma_{m \times n}}$, nos regresa un vector que representa la diagonal de esta matriz. Esto se hace por cuestiones de ahorro en la memoria, recuerde que el resto de los elementos es igual a cero.

Para poder obtener $\mathbf{\Sigma_{m \times n}}$, utilizaremos las funciones **zeros** y **fill_diagonal** de **numpy**

In [2]:
def crea_matriz_sigma(sig, num_renglones, num_columnas, aprox = False):
    '''
    ENTRADA:
    sig: Arreglo con los valores singulares
    
    num_renglones: Entero que representa el número de renglones
    
    num_columnas: Entero que representa el número de columnas
    
    aprox: Booleano, si True se crea una matriz cuadrada sigma que aproxima a nuestra matriz de datos.
    En este caso sólo se utiliza un subconjunto de los valores singulares.
    Si False, se crea una matriz rectangular utilizando todos los valores singulares
    
    SALIDA:
    matriz con la diagonal formada por sig
    '''
    if aprox:
        mat_sigma = np.zeros((num_renglones, num_renglones)) #matriz cuadrangular de ceros
    else:
        mat_sigma = np.zeros((num_renglones, num_columnas)) #matriz rectangular de ceros
    np.fill_diagonal(mat_sigma, sig) #Se modifica la diagonal
    
    return mat_sigma

In [3]:
mat_sigma = crea_matriz_sigma(sig, num_renglones, num_columnas, False)
print(mat_sigma)
print(mat_sigma.shape)

[[9.508032   0.         0.        ]
 [0.         0.77286964 0.        ]]
(2, 3)


Verifiquemos que la descomposición fue correcta

In [4]:
print('La matriz original es: ')
print(datos)
print('El producto de la descomposición es:')
print(U * mat_sigma * VT)

La matriz original es: 
[[1 2 3]
 [4 5 6]]
El producto de la descomposición es:
[[1. 2. 3.]
 [4. 5. 6.]]


## Reducción de la dimensión.
Algo que se ha observado en la práctica, es el hecho de que sólo un número, $k$, de los valores en la diagonal de la matriz $\mathbf{\Sigma_{m \times n}}$ son distintos de cero, esto indica que sólo un conjunto de $k$ atributos son considerados como importantes, los demás son ruido o repeticiones.

Así pues, podemos aproximar nuestro conjunto de datos utilizando sólomente una porción de la matriz $\mathbf{\Sigma_{m \times n}}$.

Una forma de elegir el número de valores singulares a utilizar, es estableciendo un umbral (e.g. $90\%$) para la "energía" explicada por estos valores. Esto se puede obtener sumando los cuadrados de cada valor en la diagonal ("energía" total) y considerar (agregando de mayor a menor valor) los $k$ valores singulares cuya "energía" agregada represente al menos cierto porcentaje de la energía total.

In [5]:
def selecciona_k_importantes(sig, umbral = 0.90):
    '''
    ENTRADA:
    sig: Arreglo con los valores singulares (deben de estar ordenados de mayor a menor)
    
    umbral: Número en (0,1) que representa la cantidad mínima de "energía"
    explicada por los valores singulares
    
    SALIDA:
    Un arreglo con los k valores singulares más importantes
    '''
    
    #Calcula la energía total
    total = np.sum(sig**2.0)
    
    #Calcula las proporciones acumuladas (ordenadas de mayor a menor)
    proporciones = np.cumsum(sig**2.0) / total
    
    #para almacenar los k mejores
    k_mejores = []
    
    for k in range(0, len(proporciones)):
        if proporciones[k] < umbral:
            k_mejores.append(sig[k])
        #Ya que las proporciones acumuladas se ordenan de mayor a menor
        #se sale del cíclo el primer momento en que se rebasa el umbral
        else:
            #Este k-ésimo elemento es el primero con el cual se rebasa el umbral
            k_mejores.append(sig[k]) 
            break
    #se convierte la lista en un arreglo de numpy
    k_mejores = np.array(k_mejores)
    
    return k_mejores

## Sistemas de recomendación basados en filtrado colaborativo.

¿Alguna vez se ha preguntado como Netflix, Amazon, Youtube, etc. Son capaces de realizar recomendaciones que coinciden con nuestros intereses?

Un sistema de recomendación busca predecir la valoración o preferencia que un usuario le daría a un artículo determinado (por artículo podemos referirnos a películas, canciones, libros, series accionarias, etc.)

Nosotros nos enfocaremos en sistemas de recomendación basados en filtrado colaborativo, estos sistemas funcionan comparando las opiniones o preferencias de los usuarios hacia ciertos artículos.

El filtrado colaborativo se basa en la hipótesis de que si dos usuarios tienen preferencias similares respecto a un conjunto de artículos, entonces su preferencia coincidirá para un artículo distinto. Es entonces necesario introducir el concepto de similitud.

### Funciones de similitud.
Sea $\mathbf{X}$ un conjunto. Una función $s:\mathbf{X} \times \mathbf{X} \rightarrow \mathbb{R}$ se llama una función de similitud si:

+ Es no negativa: $s(x,y) \geq 0$, para todo $x, y \in \mathbf{X}$.
+ Es simétrica: $ s(x,y) = s(y,x)$.
+ Se cumple que $s(x,y) \leq s(x, x)$ para todo $x, y \in \mathbf{X}$. Con igualdad si y sólo si $x = y$.

#### Ejemplos de funciones de similitud.
 Sean $\mathbf{X} = \mathbb{R}^n$, algunos ejemplos de funciones de similitud son los siguientes:
 
 + $cos(\mathbf{x}, \mathbf{y}) = 0.5 + 0.5 \dfrac{ < \mathbf{x}, \mathbf{y} >} {||\mathbf{x}||_{2}|| \mathbf{y}||_{2}}$ Similitud coseno (normalizada entre 0 y 1)


 
 + $euclid(\mathbf{x}, \mathbf{y}) =  \dfrac{1} { 1 + || \mathbf{x} - \mathbf{y} ||_2}$ Similitud euclideana
 
### Representación de los datos.
Usualmente, los datos se representan a través de una matriz en la cual los renglones representan los usuarios y las columnas los artículos (items).
Aquí supondremos que nuestra matriz representa la opinión que tienen 100 analistas respecto a las 10 principales series accionarias de Índice de Precios y Cotizaciones [liga](https://espanol.spindices.com/indices/equity/sp-bmv-ipc).

Cada analista emite una opinión respecto a cada una de las series accionarias:

+ $0 \implies$ todavía no se ha dado opinión.
+ $1 \implies$ opinión negativa.
+ $2 \implies$ opinión neutral.
+ $3 \implies$ opinión positiva.

De esta manera, buscamos obtener una recomendación sobre cuál serie accionaria agregar a nuestro portafolio cuando todavía no tenemos una opinión para esta.

Para simular esta matriz utilizamos la función **randint** del módulo **random** de **numpy**.

In [6]:
#Diccionario auxiliar
dicc_series = {0:'FEMSA', 1:'AMX', 2:'GFNORTE', 3:'WALMEX', 4:'GMEXICO', 5:'CEMEX', 6:'BIMBO',
           7:'GAP', 8:'ELEKTRA', 9:'TELEVISA'}
#Se fija semilla con fines de reproducir el experimento
np.random.seed(54321)
opiniones = np.random.randint(low = 0, high = 4, size = (100, 10))
print(opiniones[0:10,]) #primeras diez opiniones

[[1 2 2 0 0 2 2 2 0 3]
 [3 2 0 0 1 3 2 3 2 3]
 [0 2 1 3 1 3 1 2 0 2]
 [2 2 3 0 2 3 2 3 0 0]
 [3 0 2 2 1 0 2 1 1 1]
 [3 3 3 2 1 3 0 2 3 1]
 [1 3 0 3 2 2 1 2 0 2]
 [2 1 3 3 0 3 2 1 1 0]
 [2 0 1 2 1 2 1 1 2 3]
 [1 0 1 2 1 0 0 0 1 0]]


In [7]:
#Funciones de similitud
def cos_sim(x, y):
    '''
    Calcula la similitud coseno normalizada entre 0 y 1
    ENTRADA
    x,y: Arreglos de numpy.
    
    SALIDA
    similitud coseno entre x,y
    '''
    numerador = np.dot(x,y)
    denominador = np.linalg.norm(x, ord = 2) * np.linalg.norm(y, ord = 2)
    return 0.5 + 0.5 * (numerador / denominador)

def euclid_sim(x,y):
    '''
    Calcula la similitud euclideana
    ENTRADA
    x,y: Arreglos de numpy.
    
    SALIDA
    similitud euclideana entre x,y
    '''
    return 1.0 / (1 + np.linalg.norm(x - y))
    

Como se mencionó, el filtrado colaborativo se basa en el supuesto de que usuarios con preferencias similares para un conjunto de artículos, tendrán una preferencia similar para un nuevo artículo en consideración.

Así, supongamos que el analista $\mathbf{a_h}$ todavía no tiene una opinión para la serie accionaria $h$, en cambio, este mismo analista sí tiene una opinión para la serie accionaria $j$. Por otra parte, supongamos que existe un conjunto de analistas,  $\mathbf{b_1}, \ldots, \mathbf{b_p}$, que tienen opiniones tanto para la serie $h$ como para las serie $j$.

Con la información de estos analistas podemos crear los siguientes vectores:
$$
\mathbf{r_{h}} = 
\begin{pmatrix}
r_{b_{1}h} \\
\vdots \\
r_{b_{p}h} \\
\end{pmatrix}
$$

$$
\mathbf{r_{j}} = 
\begin{pmatrix}
r_{b_{1}j} \\
\vdots \\
r_{b_{p}j} \\
\end{pmatrix}
$$

en donde $r_{b_{i}h}$ representa la opinión que el analista $\mathbf{b_i}$ tiene sobre la serie $h$, $r_{b_{i}j}$ se define de manera similar.

Si $\mathbf{r_{h}}$ y $\mathbf{r_{j}}$ son similares (en nuestro caso esto quiere decir que la función de similitud es cercana a $1$), entonces los analistas $\mathbf{b_1}, \ldots, \mathbf{b_p}$ tienen la misma opinión entre las series $h$ y $j$, por lo tanto, para el analista $\mathbf{a_h}$, se debe asignar a la serie $h$ la misma opinión que asignó a la serie $j$ (se sigue a las multitudes).

Ya que la serie $h$ se compara con varias series $j$, lo que realmente obtenemos es una opinión promedio y no un número en el conjunto $\{0, 1, 2, 3\}$.

$$Opinion(\mathbf{a_h},h) = \dfrac{\sum_{j \in \mathbf{A}} similitud(j,h) * opinion_{\mathbf{a_h}}(j)}{\sum_{j \in \mathbf{A}} similitud(j,h)}$$

en donde

+ $Opinion(\mathbf{a_h},h)$ es la opinión del analista $\mathbf{a_h}$ para la serie $h$.
+ $similitud(j,h)$ es la similitud entre la serie $h$ y $j$, de acuerdo a los analistas que tienen opinión para ambas series.
+ $opinion_{\mathbf{a_h}}(j)$ es la opinión del analista $\mathbf{a_h}$ para la serie $j$.
+ $\mathbf{A}$ es el conjunto de analistas $\mathbf{b_1}, \ldots, \mathbf{b_p}$, es decir, aquellos analistas que tienen una opinión sobre las series $h$  y $j$.

In [8]:
def calcula_opinion(opiniones, analista, fun_sim, serie_h):
    '''
    Función para calcular la opinión (promedio) que un analista tendría para una serie accionaria
    
    ENTRADA
    opiniones: numpy.ndarray que representa las opiniones de un conjunto de analistas
    
    analista: Entero que representa el analista de interés (un índice de un renglón del arreglo opiniones)
    
    fun_sim: Función que se utilizará para calcular la similitud
    
    serie_h: Entero que representa la serie accionaria de interés (un índice de una columna del arreglo opiniones)
    
    SALIDA
    Opinión promedio de analista para la serie
    '''
    
    #Si el analista ya dio una opinión para la serie, no hay nada que hacer
    if opiniones[analista, serie_h] != 0:
        print('Para la serie ' + dicc_series[serie_h] + ' ya se tiene una opinión')
        return opiniones[analista, serie_h]
    
    #número de series accionarias
    n_series = opiniones.shape[1]
    
    #Para almacenar la similitud total y la similitud 'ponderada' por la opinión del analista
    sim_total = 0.0; sim_pond = 0.0
    
    #Para cada serie accionaria j
    for j in range(0, n_series):
        #Opinión del analista para la serie j
        opinion_serie = opiniones[analista, j]
        
        #Si el analista no tiene opinión para esta serie revisamos la siguiente
        if opinion_serie == 0: continue
        
        #Se localiza a los analistas b1,...,bp que tienen opinión para la serie_h y la serie j
        indices = np.nonzero(np.logical_and(opiniones[:, serie_h] > 0, opiniones[:, j] > 0))
        
        #Si nadie ha calificado la serie j, se establece una similitud de cero
        if len(indices) == 0:
            similitud = 0.0
        else:
            #Se calcula la similitud de acuerdo a la función fun_sim
            #Los parámetros son los vectores rh y rj
            #NOTA: opiniones[indices, serie_h] es un arreglo dentro de un arreglo, por eso se usa [0]
            similitud = fun_sim(opiniones[indices, serie_h][0], opiniones[indices, j][0])
        
        #se acumulan y se ponderan las similitudes
        sim_total = sim_total + similitud
        sim_pond = sim_pond + similitud * opinion_serie
        
    #Si no hay ninguna serie j para comparar la serie_h entonces no hay opinión
    if sim_total == 0:
        return 0
    else:
        #Opinión promedio
        return sim_pond / sim_total

In [9]:
print('Similitud coseno:')
print('Para el analista 0 en la serie', dicc_series[3], 'se tiene',calcula_opinion(opiniones, 0, cos_sim, 3))
print('Similitud euclideana:')
print('Para el analista 0 en la serie', dicc_series[3], 'se tiene',calcula_opinion(opiniones, 0, euclid_sim, 3))

Similitud coseno:
Para el analista 0 en la serie WALMEX se tiene 1.9997882689302027
Similitud euclideana:
Para el analista 0 en la serie WALMEX se tiene 1.9986945025331566


## Utilizando SVD para reducir la dimensión

Imaginemos el caso de una compañía como Netflix, cláramente, el número de usuarios rebasa al número de productos en su catálogo. En este tipo de situaciones, es conveniente reducir la dimensión de nuestros datos. Utilizando la descomposición SVD esto se realiza de la siguiente manera:

+ Elegir un número $k$ de valores singulares (por ejemplo los $k$ más grades) y obtener la matriz diagonal $\mathbf{\Sigma_{k \times k}}.$

+ Definir la matriz $\mathbf{U_{m \times k}}$ con sólamente las primeras $k$ columnas de la matriz $\mathbf{U_{m \times m}}.$

+ La matriz con la que ahora se trabajará es aquella que resulte del producto $\left(\mathbf{D_{m \times n}^{T}}\mathbf{U_{m \times k}} \mathbf{\Sigma_{k \times k}^{-1}}\right)^{T}$ (matriz de dimensión $k \times n$, $k < m$), esta matriz es igual a la submatriz formada por los primeros $k$ renglones de la matriz $\mathbf{V^T}$.

**Tarea: ¿Que matriz se utilizaría si tenemos más columnas que renglones?**.

**Tarea moral: Intente conciliar lo que se establece en este notebook con las notas de la clase**



In [10]:
def calcula_opinion_svd(opiniones, analista, fun_sim, serie_h, Vt, sig, umbral = 0.9):
    '''
    Función para calcular la opinión (promedio) que un analista tendría para una serie accionaria
    
    ENTRADA
    opiniones: numpy.ndarray que representa las opiniones de un conjunto de analistas
    
    analista: Entero que representa el analista de interés (un índice de un renglón del arreglo opiniones)
    
    fun_sim: Función que se utilizará para calcular la similitud
    
    serie_h: Entero que representa la serie accionaria de interés (un índice de una columna del arreglo opiniones)
    
    Vt: Matriz V^{T} de la descomposición SVD
    
    sig: Arreglo con los valores singulares de la descomposición SVD

    umbral: Número en (0,1) que representa la cantidad mínima de "energía"
    explicada por los valores singulares (ver función selecciona_k_importantes)
    
    SALIDA
    Opinión promedio de analista para la serie
    '''
    
    #Si el analista ya dio una opinión para la serie, no hay nada que hacer
    if opiniones[analista, serie_h] != 0:
        print('Para la serie ' + dicc_series[serie_h] + ' ya se tiene una opinión')
        return opiniones[analista, serie_h]
    
    #número de series accionarias
    n_series = opiniones.shape[1]
    
    #Para almacenar la similitud total y la similitud 'ponderada' por la opinión del analista
    sim_total = 0.0; sim_pond = 0.0
    
    #Obtiene los k valores singulares más importantes
    sig_k = selecciona_k_importantes(sig, umbral)
    k = len(sig_k)
    
    #Submatriz de la matriz V^{T}
    opiniones_svd = Vt[:k,:]
    
    #Para cada serie accionaria j
    for j in range(0, n_series):
        
        #Opinión del analista para la serie j
        opinion_serie = opiniones[analista, j]
        
        #Si el analista no tiene opinión para esta serie revisamos la siguiente
        if opinion_serie == 0: continue
        
        similitud = fun_sim(opiniones_svd[:, serie_h], opiniones_svd[:, j])
        
        #se acumulan y se ponderan las similitudes
        sim_total = sim_total + similitud
        sim_pond = sim_pond + similitud * opinion_serie
        
    #Si no hay ninguna serie j para comparar la serie_h entonces no hay opinión
    if sim_total == 0:
        return 0
    else:
        #Opinión promedio
        return sim_pond / sim_total

In [11]:
U, sig, Vt = np.linalg.svd(opiniones)
umbral = 0.9
print('Similitud coseno:')
print('Para el analista 0 en la serie',
      dicc_series[3], 'se tiene',calcula_opinion_svd(opiniones, 0, cos_sim, 3, Vt, sig, umbral ))
print('Similitud euclideana:')
print('Para el analista 0 en la serie', dicc_series[3],
      'se tiene', calcula_opinion_svd(opiniones, 0, euclid_sim, 3, Vt, sig, umbral))

Similitud coseno:
Para el analista 0 en la serie WALMEX se tiene 2.034182278191964
Similitud euclideana:
Para el analista 0 en la serie WALMEX se tiene 2.0137471636933384
